In [1]:
# ✅ 1. Install dependencies
!pip install transformers accelerate torch pdfplumber pymupdf tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ✅ 2. Imports
import os
import fitz  # PyMuPDF
import json
import re
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM


In [3]:
!pip install transformers accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 8.3 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

resume_text = "Experienced Python developer with 5 years of experience in web development..."

prompt = "Extract candidate name, experience, and skills from the following resume:\n" + resume_text

result = pipe(prompt, max_new_tokens=200)[0]['generated_text']
print(result)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


python developer with 5 years of experience in web development


In [5]:
!pip install -q pdfplumber
import pdfplumber
import sqlite3
import re
from datetime import datetime
from google.colab import files
from tqdm import tqdm


In [6]:

# ✅ 4. Upload resumes
from google.colab import files
uploaded_files = files.upload()  # Upload 1 or more PDF resumes

Saving Resume (1).pdf to Resume (1).pdf
Saving Resume (2).pdf to Resume (2).pdf
Saving Resume (3).pdf to Resume (3).pdf
Saving Resume (4).pdf to Resume (4).pdf
Saving Resume (5).pdf to Resume (5).pdf
Saving Resume (7).pdf to Resume (7).pdf
Saving Resume (8).pdf to Resume (8).pdf
Saving Resume (10).pdf to Resume (10).pdf
Saving Resume (11).pdf to Resume (11).pdf
Saving Resume (12).pdf to Resume (12).pdf
Saving Resume (13).pdf to Resume (13).pdf
Saving Resume (14).pdf to Resume (14).pdf
Saving Resume (16).pdf to Resume (16).pdf
Saving Resume (17).pdf to Resume (17).pdf
Saving Resume (18).pdf to Resume (18).pdf
Saving Resume (19).pdf to Resume (19).pdf
Saving Resume (20).pdf to Resume (20).pdf
Saving Resume (21).pdf to Resume (21).pdf
Saving Resume (26).pdf to Resume (26).pdf
Saving Resume (28).pdf to Resume (28).pdf
Saving Resume (30).pdf to Resume (30).pdf
Saving Resume (31).pdf to Resume (31).pdf
Saving Resume (34).pdf to Resume (34).pdf
Saving Resume (35).pdf to Resume (35).pdf
Saving

In [8]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text


In [9]:
file_name = list(uploaded_files.keys())[0]
text = extract_text_from_pdf(file_name)
print(text[:1000])


DR. SWAPNIL MORANDE
AI/ML Researcher | Data Scientist | GenAI Practitioner with expertise in LLMs and Information Systems,
with a proven track record in the AI/ML domain and publications in leading academic journals.
AI IMPACT SUMMARY
ü Scale: 8+ production ML models deployed
ü Innovation: Contributed to 2 patents/frameworks
ü Technical Depth: 3 deployments of GenAI/LLM projects
PROFILE
P
h.D. under the National Operating Program of the EU, with specialization in AI/ML & Data Science, and Impact
MBA (USA), with significant contributions to research across diverse sectors. A dedicated academic actively
engaged in advancing computational intelligence through innovative and cutting-edge studies and experiments.
KEY COMPETENCIES
# DATA SCIENCE # DESIGN THINKING
# AI/ML # GENERATIVE AI & LLMs
# PROJECT MANAGEMENT # BUSINESS ANALYSIS
CAREER HIGHLIGHTS
Ø Titled 'Doctor Europaeus' by the European University Association (EUA) for pioneering machine learning
modelling in healthcare, significantl

In [10]:
import sqlite3

# Connect to SQLite database (it will create if not exists)
conn = sqlite3.connect("GoogleflanParsed.db")
cursor = conn.cursor()

# Drop old table if needed (WARNING: This deletes old data)
cursor.execute("DROP TABLE IF EXISTS resumes")

# Create table with correct columns
cursor.execute("""
CREATE TABLE IF NOT EXISTS resumes (
    resume_path TEXT PRIMARY KEY,
    person_name TEXT,
    current_role TEXT,
    total_experience TEXT,
    current_tenure TEXT,
    certification_count INTEGER,
    publication_count INTEGER,
    skill_density REAL,
    full_parsed_json TEXT,
    created_at TEXT DEFAULT CURRENT_TIMESTAMP
)
""")

conn.commit()
conn.close()
print("Table created with correct columns.")


Table created with correct columns.


In [11]:
import pdfplumber
import sqlite3
from datetime import datetime

def extract_text_from_pdf(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() or ""
    return text.strip()

conn = sqlite3.connect("GoogleflanParsed.db")
cursor = conn.cursor()

for file_name in uploaded_files.keys():
    print(f"Processing {file_name} ...")
    extracted_text = extract_text_from_pdf(file_name)

    # We'll store extracted text inside 'full_parsed_json' for now
    cursor.execute("""
        INSERT OR REPLACE INTO resumes (
            resume_path, person_name, current_role, total_experience, current_tenure,
            certification_count, publication_count, skill_density, full_parsed_json, created_at
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        file_name,          # resume_path
        None,               # person_name
        None,               # current_role
        None,               # total_experience
        None,               # current_tenure
        None,               # certification_count
        None,               # publication_count
        None,               # skill_density
        extracted_text,     # full_parsed_json - storing raw text for now
        datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    ))
    conn.commit()
    print(f"Inserted {file_name} into DB.")

conn.close()
print("Done inserting all resumes.")


Processing Resume (1).pdf ...
Inserted Resume (1).pdf into DB.
Processing Resume (2).pdf ...
Inserted Resume (2).pdf into DB.
Processing Resume (3).pdf ...
Inserted Resume (3).pdf into DB.
Processing Resume (4).pdf ...
Inserted Resume (4).pdf into DB.
Processing Resume (5).pdf ...
Inserted Resume (5).pdf into DB.
Processing Resume (7).pdf ...
Inserted Resume (7).pdf into DB.
Processing Resume (8).pdf ...
Inserted Resume (8).pdf into DB.
Processing Resume (10).pdf ...
Inserted Resume (10).pdf into DB.
Processing Resume (11).pdf ...
Inserted Resume (11).pdf into DB.
Processing Resume (12).pdf ...
Inserted Resume (12).pdf into DB.
Processing Resume (13).pdf ...
Inserted Resume (13).pdf into DB.
Processing Resume (14).pdf ...
Inserted Resume (14).pdf into DB.
Processing Resume (16).pdf ...
Inserted Resume (16).pdf into DB.
Processing Resume (17).pdf ...
Inserted Resume (17).pdf into DB.
Processing Resume (18).pdf ...
Inserted Resume (18).pdf into DB.
Processing Resume (19).pdf ...
Inserted

Inserted Resume (21).pdf into DB.
Processing Resume (26).pdf ...
Inserted Resume (26).pdf into DB.
Processing Resume (28).pdf ...
Inserted Resume (28).pdf into DB.
Processing Resume (30).pdf ...
Inserted Resume (30).pdf into DB.
Processing Resume (31).pdf ...
Inserted Resume (31).pdf into DB.
Processing Resume (34).pdf ...
Inserted Resume (34).pdf into DB.
Processing Resume (35).pdf ...
Inserted Resume (35).pdf into DB.
Processing Resume (38).pdf ...
Inserted Resume (38).pdf into DB.
Processing Resume (41).pdf ...
Inserted Resume (41).pdf into DB.
Processing Resume (45).pdf ...
Inserted Resume (45).pdf into DB.
Processing Resume (47).pdf ...
Inserted Resume (47).pdf into DB.
Processing Resume (50).pdf ...
Inserted Resume (50).pdf into DB.
Processing Resume (51).pdf ...
Inserted Resume (51).pdf into DB.
Processing Resume (52).pdf ...
Inserted Resume (52).pdf into DB.
Processing Resume (53).pdf ...
Inserted Resume (53).pdf into DB.
Processing Resume (54).pdf ...
Inserted Resume (54).pdf 

Inserted Resume (74).pdf into DB.
Processing Resume (75).pdf ...
Inserted Resume (75).pdf into DB.
Processing Resume (76).pdf ...
Inserted Resume (76).pdf into DB.
Processing Resume (78).pdf ...
Inserted Resume (78).pdf into DB.
Processing Resume (79).pdf ...
Inserted Resume (79).pdf into DB.
Processing Resume (80).pdf ...
Inserted Resume (80).pdf into DB.
Processing Resume (81).pdf ...
Inserted Resume (81).pdf into DB.
Processing Resume (82).pdf ...
Inserted Resume (82).pdf into DB.
Processing Resume (83).pdf ...
Inserted Resume (83).pdf into DB.
Processing Resume (84).pdf ...
Inserted Resume (84).pdf into DB.
Processing Resume (85).pdf ...
Inserted Resume (85).pdf into DB.
Processing Resume (86).pdf ...
Inserted Resume (86).pdf into DB.
Processing Resume (87).pdf ...
Inserted Resume (87).pdf into DB.
Processing Resume (88).pdf ...
Inserted Resume (88).pdf into DB.
Processing Resume (89).pdf ...
Inserted Resume (89).pdf into DB.
Processing Resume (91).pdf ...
Inserted Resume (91).pdf 

Inserted Resume (108).pdf into DB.
Processing Resume (110).pdf ...


Inserted Resume (110).pdf into DB.
Processing Resume (111).pdf ...
Inserted Resume (111).pdf into DB.
Processing Resume (112).pdf ...
Inserted Resume (112).pdf into DB.
Processing Resume (114).pdf ...
Inserted Resume (114).pdf into DB.
Processing Resume (115).pdf ...
Inserted Resume (115).pdf into DB.
Processing Resume (116).pdf ...
Inserted Resume (116).pdf into DB.
Processing Resume (118).pdf ...
Inserted Resume (118).pdf into DB.
Processing Resume (119).pdf ...
Inserted Resume (119).pdf into DB.
Processing Resume (120).pdf ...
Inserted Resume (120).pdf into DB.
Processing Resume (122).pdf ...
Inserted Resume (122).pdf into DB.
Processing Resume (124).pdf ...
Inserted Resume (124).pdf into DB.
Processing Resume (125).pdf ...
Inserted Resume (125).pdf into DB.
Processing Resume (126).pdf ...
Inserted Resume (126).pdf into DB.
Processing Resume (127).pdf ...
Inserted Resume (127).pdf into DB.
Processing Resume (128).pdf ...
Inserted Resume (128).pdf into DB.
Processing Resume (129).pdf

Inserted Resume (143).pdf into DB.
Processing Resume (146).pdf ...
Inserted Resume (146).pdf into DB.
Processing Resume (147).pdf ...


Inserted Resume (147).pdf into DB.
Done inserting all resumes.


In [12]:
import sqlite3

conn = sqlite3.connect("/content/GoogleflanParsed.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM resumes LIMIT 5")  # Get first 5 rows for preview
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()



('Resume (1).pdf', None, None, None, None, None, None, None, 'DR. SWAPNIL MORANDE\nAI/ML Researcher | Data Scientist | GenAI Practitioner with expertise in LLMs and Information Systems,\nwith a proven track record in the AI/ML domain and publications in leading academic journals.\nAI IMPACT SUMMARY\nü Scale: 8+ production ML models deployed\nü Innovation: Contributed to 2 patents/frameworks\nü Technical Depth: 3 deployments of GenAI/LLM projects\nPROFILE\nP\nh.D. under the National Operating Program of the EU, with specialization in AI/ML & Data Science, and Impact\nMBA (USA), with significant contributions to research across diverse sectors. A dedicated academic actively\nengaged in advancing computational intelligence through innovative and cutting-edge studies and experiments.\nKEY COMPETENCIES\n# DATA SCIENCE # DESIGN THINKING\n# AI/ML # GENERATIVE AI & LLMs\n# PROJECT MANAGEMENT # BUSINESS ANALYSIS\nCAREER HIGHLIGHTS\nØ Titled \'Doctor Europaeus\' by the European University Associ

In [13]:
import re
from datetime import datetime
from dateutil import parser

def extract_person_name(text):
    lines = text.strip().split('\n')
    for line in lines:
        line = line.strip()
        # Simple heuristic: name is a line with 2-4 words and mostly alphabets (not email/phone)
        if 2 <= len(line.split()) <= 4 and all(c.isalpha() or c.isspace() for c in line):
            return line
    return None

def extract_current_role(text):
    role_keywords = ['Researcher', 'Engineer', 'Scientist', 'Analyst', 'Developer', 'Manager', 'Consultant', 'Intern']
    lines = text.strip().split('\n')
    for line in lines[:10]:  # check first 10 lines
        for role in role_keywords:
            if role.lower() in line.lower():
                return line.strip()
    return None

def extract_total_experience(text):
    # Try to find explicit years of experience patterns
    patterns = [
        r'(\d+(\.\d+)?)\s*\+?\s*(years|yrs)',  # e.g. 3+ years
        r'(over|more than|approximately|about)\s*(\d+(\.\d+)?)\s*(years|yrs)',
        r'experience\s*(of)?\s*(\d+(\.\d+)?)\s*(years|yrs)',
    ]
    for pat in patterns:
        matches = re.findall(pat, text, re.I)
        if matches:
            for match in matches:
                # The number might be in different groups depending on pattern, so check each group
                for g in match:
                    try:
                        val = float(g)
                        return val
                    except:
                        pass

    # If no explicit mention, try to infer from date ranges (e.g. Jan 2018 - Present)
    total_months = 0
    date_ranges = re.findall(r'([A-Za-z]{3,9}\s?\d{4})\s*[-–]\s*(Present|Now|[\w\s]{3,9}\s?\d{4})', text)
    for start_str, end_str in date_ranges:
        try:
            start_date = parser.parse(start_str)
            end_date = datetime.now() if end_str.lower() in ['present', 'now'] else parser.parse(end_str)
            diff = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
            if diff > 0:
                total_months += diff
        except:
            continue
    years = total_months / 12
    return round(years, 1) if years > 0 else None

def extract_current_tenure(text):
    # Search for lines mentioning current role or tenure and parse date ranges
    lines = text.split('\n')
    for line in lines:
        if re.search(r'(current role|present role|currently working|current tenure)', line, re.I):
            dates = re.findall(r'([A-Za-z]{3,9}\s?\d{4})\s*[-–]\s*(Present|Now|[\w\s]{3,9}\s?\d{4})', line)
            if dates:
                try:
                    start_str, end_str = dates[0]
                    start_date = parser.parse(start_str)
                    end_date = datetime.now() if end_str.lower() in ['present', 'now'] else parser.parse(end_str)
                    diff = (end_date.year - start_date.year) + (end_date.month - start_date.month) / 12
                    return round(diff, 1)
                except:
                    continue
    # fallback: look for "current tenure" nearby a number of years
    match = re.search(r'current tenure.*?(\d+(\.\d+)?)\s*(years|yrs)', text, re.I)
    if match:
        return float(match.group(1))
    return None

def extract_certification_count(text):
    count = len(re.findall(r'certificat', text, re.I))
    return count

def extract_publication_count(text):
    count = len(re.findall(r'publicat|paper', text, re.I))
    return count

def extract_skill_density(text):
    skills = ['python', 'java', 'c++', 'machine learning', 'deep learning', 'nlp', 'sql', 'r', 'tensorflow', 'pytorch', 'excel', 'tableau']
    count = 0
    text_lower = text.lower()
    for skill in skills:
        if skill in text_lower:
            count += 1
    return count


In [14]:
import sqlite3
import csv
from datetime import datetime

# Assuming conn and cursor are your SQLite connection and cursor
conn = sqlite3.connect('GoogleflanParsed.db')
cursor = conn.cursor()

# Create a new table for extracted features if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS resume_features (
    resume_path TEXT PRIMARY KEY,
    name TEXT,
    current_role TEXT,
    total_experience REAL,
    current_tenure REAL,
    certification_count INTEGER,
    publication_count INTEGER,
    skill_density INTEGER,
    updated_at TEXT
)
""")
conn.commit()

# Fetch all resumes from your resumes table
cursor.execute("SELECT resume_path, full_parsed_json FROM resumes")
all_resumes = cursor.fetchall()


results = []

for resume_path, text in all_resumes:
    if text:
        name = extract_person_name(text)
        current_role = extract_current_role(text)
        total_experience = extract_total_experience(text)
        current_tenure = extract_current_tenure(text)
        certification_count = extract_certification_count(text)
        publication_count = extract_publication_count(text)
        skill_density = extract_skill_density(text)

        updated_at = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Insert or replace into resume_features
        cursor.execute("""
        INSERT OR REPLACE INTO resume_features
        (resume_path, name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density, updated_at)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (resume_path, name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density, updated_at))

        results.append((resume_path, name, current_role, total_experience, current_tenure, certification_count, publication_count, skill_density))

conn.commit()

# Optional: write to CSV
with open('resume_features.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Resume Path', 'Name', 'Current Role', 'Total Experience', 'Current Tenure', 'Certification Count', 'Publication Count', 'Skill Density'])
    writer.writerows(results)

print("Extraction completed and saved to database and CSV.")


Extraction completed and saved to database and CSV.


In [15]:
import sqlite3
from tabulate import tabulate  # optional, pip install tabulate

conn = sqlite3.connect('GoogleflanParsed.db')
cursor = conn.cursor()

cursor.execute("SELECT * FROM resume_features")
rows = cursor.fetchall()

# Fetch column names
col_names = [description[0] for description in cursor.description]

print(tabulate(rows, headers=col_names, tablefmt='psql'))

conn.close()


+------------------+---------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+------------------+-----------------------+---------------------+-----------------+---------------------+
| resume_path      | name                                  | current_role                                                                                                                                    |   total_experience | current_tenure   |   certification_count |   publication_count |   skill_density | updated_at          |
|------------------+---------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------+--------------------+------------------+-----------------------+---------------------+-----------------+---------------------|
|

In [22]:
import sqlite3

conn = sqlite3.connect("/content/GoogleflanParsed.db")
cursor = conn.cursor()

# Show all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("📋 Tables in DB:", tables)
conn.close()


📋 Tables in DB: [('resumes',), ('resume_features',)]
